### Contents:
#### 1. Import data and libraries
#### 2. Data wrangling
#### 3. Data consistency checks
#### 4. Plotting a choropleth
    Map of total order value
    Map of total number of orders
    Map of average order value
#### 5. Analysing results

### 1. Import data and libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
from folium import plugins
from folium.features import GeoJsonTooltip
import json

In [2]:
# This command propts matplotlib visuals to appear in the notebook 

%matplotlib inline

In [3]:
# Import JSON
country_geo = (f"C:/Users/ruthf/OneDrive/Documents/Data Immersion/Achievement 6/Project/02 Data/1. Orignal Data/us-states.json")

In [4]:
# That's just in case you want to look at the JSON file contents here too:

f = open(r'C:/Users/ruthf/OneDrive/Documents/Data Immersion/Achievement 6/Work Alongs/Data/us-states.json',)
  
# returns JSON object asa dictionary
data = json.load(f)
  
# Iterating through the json list
for i in data['features']:
    print(i)

{'type': 'Feature', 'id': 'AL', 'properties': {'name': 'Alabama'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-87.359296, 35.00118], [-85.606675, 34.984749], [-85.431413, 34.124869], [-85.184951, 32.859696], [-85.069935, 32.580372], [-84.960397, 32.421541], [-85.004212, 32.322956], [-84.889196, 32.262709], [-85.058981, 32.13674], [-85.053504, 32.01077], [-85.141136, 31.840985], [-85.042551, 31.539753], [-85.113751, 31.27686], [-85.004212, 31.003013], [-85.497137, 30.997536], [-87.600282, 30.997536], [-87.633143, 30.86609], [-87.408589, 30.674397], [-87.446927, 30.510088], [-87.37025, 30.427934], [-87.518128, 30.280057], [-87.655051, 30.247195], [-87.90699, 30.411504], [-87.934375, 30.657966], [-88.011052, 30.685351], [-88.10416, 30.499135], [-88.137022, 30.318396], [-88.394438, 30.367688], [-88.471115, 31.895754], [-88.241084, 33.796253], [-88.098683, 34.891641], [-88.202745, 34.995703], [-87.359296, 35.00118]]]}}
{'type': 'Feature', 'id': 'AK', 'properties': {'name': 'Alaska'},

In [5]:
country_geo

'C:/Users/ruthf/OneDrive/Documents/Data Immersion/Achievement 6/Project/02 Data/1. Orignal Data/us-states.json'

In [6]:
# Import StockX data

df = pd.read_excel(r"C:\Users\ruthf\OneDrive\Documents\Data Immersion\Achievement 6\Project\02 Data\1. Orignal Data\StockX_data_add_columns.xlsx")

In [7]:
df.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region,Profit,Profit %,Release Year,Sold Year,Days since release
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California,877.0,3.986364,2016,2017,342
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California,465.0,2.113636,2016,2017,282
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220,2016-11-23,11.0,California,470.0,2.136364,2016,2017,282
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220,2016-11-23,11.5,Kentucky,855.0,3.886364,2016,2017,282
4,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828.0,220,2017-02-11,11.0,Rhode Island,608.0,2.763636,2017,2017,202


In [8]:
df.shape

(99956, 13)

### 2. Data Wrangling

In [9]:
# Create series with the columns and aggregation needed to plot map
data_to_map = df.groupby('Buyer Region', as_index=False).agg({"Sale Price": "sum","Order Date":"count"})

In [10]:
# Change columns to more readable title

data_to_map.rename(columns = {'Sale Price': 'Total Order Value','Order Date':'Total Orders'},inplace = True)

In [11]:
data_to_map.head()

,Buyer Region,Total Order Value,Total Orders
0,Alabama,1.809070e+05,457
1,Alaska,2.656000e+04,61
2,Arizona,6.278063e+05,1398
3,Arkansas,7.056000e+04,173
4,California,9.253625e+06,19349


In [12]:
data_to_map.dtypes

Buyer Region          object
Total Order Value    float64
Total Orders           int64
dtype: object

In [13]:
# View data to map and apply number formatting
pd.options.display.float_format = '{:,.2f}'.format
data_to_map.sort_values(by = 'Total Order Value', ascending=False)

,Buyer Region,Total Order Value,Total Orders
4,California,"9,253,624.87",19349
32,New York,"7,316,726.73",16525
37,Oregon,"3,606,620.79",7681
9,Florida,"2,840,033.66",6376
43,Texas,"2,451,756.04",5876
30,New Jersey,"2,118,430.84",4720
13,Illinois,"1,623,696.91",3782
38,Pennsylvania,"1,310,371.21",3109
21,Massachusetts,"1,248,750.99",2903
22,Michigan,"1,154,573.62",2762


In [14]:
# Check dtypes as some ways of formatting convert float to object
data_to_map.dtypes

Buyer Region          object
Total Order Value    float64
Total Orders           int64
dtype: object

In [15]:
# Check that Buyer region column can be called to plot the map.
data_to_map.columns

Index(['Buyer Region', 'Total Order Value', 'Total Orders'], dtype='object')

In [16]:
data_to_map.shape

(51, 3)

### 3. Data consistency checks

In [17]:
# Check for missing values

data_to_map.isnull().sum()

Buyer Region         0
Total Order Value    0
Total Orders         0
dtype: int64

In [18]:
# Check for duplicates
data_to_map[data_to_map.duplicated()]

,Buyer Region,Total Order Value,Total Orders


### 4. Plotting a choropleth

#### Map of total order value

In [19]:
# Setup a folium map at a high-level zoom 40 -95 brings US to the centre
map = folium.Map(location = [40, -95], zoom_start = 4)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_map,
    columns = ['Buyer Region', 'Total Order Value'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'YlGn', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Total Order Value").add_to(map)
folium.LayerControl().add_to(map)

map

In [24]:
# Save map
map.save('sales_map.html')

#### Map of total number of orders

In [20]:
### Plot number of orders

# Setup a folium map at a high-level zoom
map = folium.Map(location = [40, -95], zoom_start = 4)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_map,
    columns = ['Buyer Region', 'Total Orders'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'YlGn', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Number of Orders").add_to(map)
folium.LayerControl().add_to(map)

map

In [28]:
# Save map
map.save('orders_map.html')

#### Map of average sale price

In [21]:
# Create df.  Alternative method to creating series where df columns can be indexed.
data_3=df.groupby(['Buyer Region'])['Sale Price'].mean().reset_index()

data_3

,Buyer Region,Sale Price
0,Alabama,395.86
1,Alaska,435.41
2,Arizona,449.07
3,Arkansas,407.86
4,California,478.25
5,Colorado,434.02
6,Connecticut,412.42
7,Delaware,503.34
8,District of Columbia,454.23
9,Florida,445.43


In [22]:
state_data_indexed = data_3.set_index('Buyer Region')
state_data_indexed.head()

,Sale Price
Buyer Region,
Alabama,395.86
Alaska,435.41
Arizona,449.07
Arkansas,407.86
California,478.25


In [23]:
data_3.columns

Index(['Buyer Region', 'Sale Price'], dtype='object')

In [24]:
# Plot map average order value
# Setup a folium map at a high-level zoom
map = folium.Map(location = [40, -95], zoom_start = 4)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
cp = folium.Choropleth(
    geo_data = country_geo, 
    data = data_3,
    columns = ['Buyer Region', 'Sale Price'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'YlGn', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Average Sale Price").add_to(map)
folium.LayerControl().add_to(map)
# Adding labels to map
for s in cp.geojson.data['features']:
    if s['properties']['name'] in state_data_indexed.index:
       s['properties']['Sale Price'] = state_data_indexed.loc[s['properties']['name'],'Sale Price']
    folium.GeoJsonTooltip(['name','Sale Price']).add_to(cp.geojson)

map

In [33]:
# Save map
map.save('avg_orders_map.html')

### 5. Analysing Results

#### Does the analysis answer any of your existing research questions?
#### One of my initial questions was: Are there any trends in regional buying habits?

#### The maps show that buyers in California place considerably more orders and generate the highest sales value for the platform.  Followed by New York.  This is to be expected as both states have a prolific streetwear and sneaker scene and culture.  Other states such as Oregon, Texas, Florida, Ohio and New Jersey also generate a high number of orders and sales value.  This is likely driven by the major cities in the states such as Portland, Dallas and Chicago.  The remaining states all perform relitavely equally.

#### Interestingly when looking at average sale price it appears California and Nevada are willing to spend the highest amount on a pair of sneakers.  Interestingly this is less in New York and Texas, but could indicate that buyers have more options and access to alternative methods of buying sneakers than using the StockX platform.  Whereas buyers in more rural states such as Vermont, Iowa and Kentucky are willing to pay a premium to obtain premium sneakers as they have less local methods to purchase from.  Wyoming and West Virginia have the lowest average order value indicatng buyers here are more price concious.

#### Does the analysis lead you to any new research questions?

#### It would be interesting to see these trends over time to see if there has been any shift in the market in terms of state buying habits.
#### I would also like to look at different variables to understand which brand or sneaker is most popular in each state.